In [ ]:
import pandas as pd
import streamlit as st
import warnings

# Ignorar o warning de SettingWithCopy
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

st.set_page_config(page_title="Análise Preditiva de Clientes", layout="centered")

st.title("🔎 Máquina Preditiva de Clientes")

# Carregar os dados
@st.cache_data
def carregar_dados(caminho):
    return pd.read_excel(caminho, sheet_name='DADOS PREDITIVA')

file_path = r"C:/Kidy/PREDITIVA/DADOS_PREDITIVA.xlsx"  # ajuste o caminho conforme necessário
df = carregar_dados(file_path)

# Interface
st.sidebar.header("Filtros de Cliente")
codigo_cliente = st.sidebar.text_input("Código do Cliente", "")
codigo_grupo_cliente = st.sidebar.text_input("Código do Grupo Cliente", "")

# Função principal
def analisar_cliente(codigo_cliente=None, codigo_grupo_cliente=None):
    # Filtro por cliente ou grupo de cliente
    if codigo_cliente:
        codigo_cliente = int(codigo_cliente)
        dados_cliente = df[df['Codigo Cliente'] == codigo_cliente].copy()
    elif codigo_grupo_cliente:
        dados_cliente = df[df['Codigo Grupo Cliente'] == codigo_grupo_cliente].copy()
    else:
        st.warning("Informe um código de cliente ou código de grupo de cliente!")
        return

    if dados_cliente.empty:
        st.error("Nenhum dado encontrado para o código informado!")
        return
    
    # Pegar o nome do cliente (ou grupo) e data da última compra
    nome_cliente = dados_cliente['Razao Social'].iloc[0] if codigo_cliente else dados_cliente['Grupo Cliente'].iloc[0]
    ultima_compra = dados_cliente['Data Ultima Compra'].max().strftime('%d/%m/%Y')
    
    st.subheader(f"📌 Cliente: {nome_cliente}")
    st.write(f"🗓️  Última Compra: {ultima_compra}")
    
    # Criar uma coluna com o mês da data de cadastro
    dados_cliente.loc[:, 'Mes Cadastro'] = dados_cliente['Data Cadastro'].dt.month
    
    # Analisar as datas de TODOS os pedidos (não só o cadastro)
    dados_cliente.loc[:, 'Mes Pedido'] = dados_cliente['Data Cadastro'].dt.month
    
    # Melhor mês com base na frequência dos pedidos realizados
    melhor_mes = dados_cliente['Mes Pedido'].mode()[0]
    
    st.info(f"👉 Melhor mês para oferecer produtos: {melhor_mes}")

    # Análise das 10 melhores linhas de produto
    top_linhas = (dados_cliente.groupby('Linha')['Qtd Venda']
                  .sum()
                  .sort_values(ascending=False)
                  .head(10))
    
    st.subheader("👉 Top 10 Linhas de Produtos para Oferecer:")
    st.table(top_linhas.reset_index().rename(columns={'Qtd Venda': 'Unidades Vendidas'}))

# Botão para rodar a análise
if st.sidebar.button("🔍 Analisar Cliente"):
    analisar_cliente(codigo_cliente, codigo_grupo_cliente)
